# Prepping COCA
In this notebook, I'll prepare relevant COCA sections to train the Roark Parser.
## Table of Contents
1. [Loading COCA](#1)
2. [Test Run](#2)
3. [Full Run](#3)
<a id='1'>

## Loading COCA

In [2]:
import os
root = 'D:/COCA/stim/text_newspaper_lsp/'
text = []
for fn in os.listdir(root):
    print("opening file", fn)
    f = open(root + fn)
    lines = f.readlines()
    text.append(lines)
    f.close()
len(text)

opening file w_news_1999.txt
opening file w_news_1994.txt
opening file w_news_2005.txt
opening file w_news_1993.txt
opening file w_news_2012.txt
opening file w_news_2004.txt
opening file w_news_1995.txt
opening file w_news_2009.txt
opening file w_news_1997.txt
opening file w_news_2007.txt
opening file w_news_2006.txt
opening file w_news_1991.txt
opening file w_news_1996.txt
opening file w_news_2000.txt
opening file w_news_2003.txt
opening file w_news_1990.txt
opening file w_news_2011.txt
opening file w_news_2001.txt
opening file w_news_2002.txt
opening file w_news_1998.txt
opening file w_news_2008.txt
opening file w_news_1992.txt
opening file w_news_2010.txt


23

<a id='2'>

## Test Run

In [3]:
import nltk
mini = text[:5]

In [26]:
mini[0][1]

"##3000589 <p> Some teams improve by adding players . Others make themselves better by getting rid of players . Addition by subtraction , Branch Rickey called it . Then there are the Yankees , who hope to prove that a team that won more games than any other in baseball history can at least repeat as champions , if not match or exceed that victory total , without making any changes . <p> When the Yankees ' pitchers and catchers have their first spring training workout in Tampa , Fla. , on Thursday and when the rest of the World Series winners join them Feb. 23 , they will not have to introduce themselves to each other . They are the same guys who dressed and played together last season . <p> True , Tim Raines will not be there , but he is the lone exception . Every other member of the post-season team will be back in all the old familiar places . Bernie Williams , who seemed certain to leave , stayed when the numbers after the dollar sign got big enough . David Cone could @ @ @ @ @ @ @ 

In [45]:
out = []
for line_chunks in mini:
    for line_chunk in line_chunks:
        sents = nltk.sent_tokenize(line_chunk)
        sentsList = []
        for sent in sents:
            if '@' in sent:
                continue
            if '<p>' in sent:
                sent = sent.split('<p>')[1]
            sentsList.append(sent)
        out.append(sentsList)

In [46]:
len(out)
foo_mini = out[1]

In [47]:
import pandas as pd
from stanfordcorenlp import StanfordCoreNLP
from nltk import StanfordPOSTagger
from nltk.parse import stanford
from nltk.parse import CoreNLPParser

In [48]:
parser = CoreNLPParser(url='http://localhost:9000')

In [49]:
coca_trees = []
for sent in foo_mini:
    trees = parser.raw_parse(sent)
    for tree in trees:
        coca_trees.append(tree)
len(coca_trees)

60

In [56]:
coca_trees[0].pretty_print()

                       ROOT                           
                        |                              
                        S                             
       _________________|___________________________   
      |                      VP                     | 
      |            __________|____                  |  
      |           |               PP                | 
      |           |      _________|_____            |  
      |           |     |               S           | 
      |           |     |               |           |  
      |           |     |               VP          | 
      |           |     |          _____|_____      |  
      NP          |     |         |           NP    | 
  ____|____       |     |         |           |     |  
 DT       NNS    VBP    IN       VBG         NNS    . 
 |         |      |     |         |           |     |  
Some     teams improve  by      adding     players  . 



<a id='3'>

## Full Run

In [84]:
def tokenizeAndParseSents(coca_sect):
    sent_chunks = []
    for line_chunks in coca_sect:
        for line_chunk in line_chunks:
            stoks = nltk.sent_tokenize(line_chunk)
            slist = []
            for sent in stoks:
                if '@' in sent:
                    continue
                if '<p>' in sent:
                    sent = sent.split('<p>')[1]
                slist.append(sent)
            sent_chunks.append(slist)
    sent_chunks.remove(sent_chunks[0]) #it's a '/n'
    #sent_tokenization complete
    parses = []
    for sents in sent_chunks:
        for sent in sents:
            trees = parser.raw_parse(sent)
            for tree in trees:
                parses.append(tree)
    return parses

In [87]:
%time test_parses = tokenizeAndParseSents(mini[:2])

StopIteration: 

In [88]:
test_parses[0]

[' Some teams improve by adding players .',
 'Others make themselves better by getting rid of players .',
 'Addition by subtraction , Branch Rickey called it .',
 'Then there are the Yankees , who hope to prove that a team that won more games than any other in baseball history can at least repeat as champions , if not match or exceed that victory total , without making any changes .',
 " When the Yankees ' pitchers and catchers have their first spring training workout in Tampa , Fla. , on Thursday and when the rest of the World Series winners join them Feb. 23 , they will not have to introduce themselves to each other .",
 'They are the same guys who dressed and played together last season .',
 ' True , Tim Raines will not be there , but he is the lone exception .',
 'Every other member of the post-season team will be back in all the old familiar places .',
 'Bernie Williams , who seemed certain to leave , stayed when the numbers after the dollar sign got big enough .',
 " General Mana